# RRAP-IS Authentication Demo Notebook

> A tutorial of RRAP authentication using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, authentication]

## About

This notebook is a demonstration of authenticating with the RRAP-IS system.  

### Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [9]:
import requests
import os
import sys
import json
from IPython.core.display import display, HTML
from IPython.display import IFrame
from mdsisclienttools.auth.TokenManager import DeviceFlowManager

/tmp/ipykernel_19616/723406562.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [10]:
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display(f'Checking base urls')

for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed


## Authentication

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

[Return to Top](#toc)

In [11]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

Using storage type: FILE.
Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Trying to use found tokens to refresh the access token.

Tokens found in storage but they are not valid.

Initiating device auth flow to setup offline access token.

Decoding response

Please authorise using the following endpoint.

Verification URL: https://auth.dev.rrap-is.com/auth/realms/rrap/device?user_code=HVXR-VLRW
User Code: HVXR-VLRW

Awaiting completion


Token generation complete. Authorisation successful.



## Endpoint Documentation
Endpoint documentation can be found by appending either `/docs` or `/redoc` on the end a base URL.

For example:
<ul>
  <li><a href="https://registry-api.testing.rrap-is.com/redoc#tag/Access-check" target="_blank">Redoc FastAPI - Access-check</a></li>
  <li><a href="https://registry-api.testing.rrap-is.com/docs#/Access%20check" target="_blank">Docs FastAPI - Access-check</a> </li>
</ul>

Then select from the menu an endpoint function call e.g. `/check-access/check-general-access`

Then append the function call onto the base url e.g. `https://registry-api.testing.rrap-is.com/check-access/check-general-access`

[Return to Top](#toc)

## Testing

### Test general access 

In [18]:
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-general-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIzMVBQVkZZdE53a2JEcVZnNjFqX08xNXdiN2FRZU1BaExCbWdpY2VvY2dBIn0.eyJleHAiOjE2NzE0MjgyMzUsImlhdCI6MTY3MTQyNzkzNSwiYXV0aF90aW1lIjoxNjcxNDI3Mjk0LCJqdGkiOiIxYTRlZTM2My0zNzUwLTQ1YjItOWY5OC04YTE0ZjliM2UwMTUiLCJpc3MiOiJodHRwczovL2F1dGguZGV2LnJyYXAtaXMuY29tL2F1dGgvcmVhbG1zL3JyYXAiLCJhdWQiOlsicmVhbG0tbWFuYWdlbWVudCIsImFjY291bnQiXSwic3ViIjoiYWRjMDRiYzItNTZiOS00YzFhLTk0MTktMGE2ZTcxYTU5Zjg2IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY2xpZW50LXRvb2xzIiwic2Vzc2lvbl9zdGF0ZSI6IjgyMTFkNTBlLWQ5YTQtNGNiZC1iYmVjLWQ2MzAwYTQ4Njg4NiIsImFjciI6IjAiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsic2VhcmNoLXJlYWQiLCJlbnRpdHktcmVnaXN0cnktYWRtaW4iLCJkYXRhLXN0b3JlLXdyaXRlIiwic3lzLWFkbWluLXJlYWQiLCJwcm92LWFkbWluIiwiaGFuZGxlLXdyaXRlIiwic3lzLWFkbWluLXdyaXRlIiwiaGFuZGxlLWFkbWluIiwicHJvdi1yZWFkIiwiZW50aXR5LXJlZ2lzdHJ5LXdyaXRlIiwiZGVmYXVsdC1yb2xlcy1ycmFwIiwicHJvdi13cml0ZSIsImVudGl0eS1yZWdpc3RyeS1yZWFkIiwib2ZmbGluZV9hY2Nlc3MiLCJkYXRhLXN0b3JlLWFkbWluIiwiZGF0YS1zdG9yZS1yZWFkIiwia

### Test read access

In [19]:
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-read-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json()['roles'], indent=4, sort_keys=True))

[
    "search-read",
    "entity-registry-admin",
    "data-store-write",
    "sys-admin-read",
    "prov-admin",
    "handle-write",
    "sys-admin-write",
    "handle-admin",
    "prov-read",
    "entity-registry-write",
    "default-roles-rrap",
    "prov-write",
    "entity-registry-read",
    "offline_access",
    "data-store-admin",
    "data-store-read",
    "handle-read",
    "uma_authorization",
    "sys-admin-admin"
]


### Test write access

In [20]:
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-write-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json()['roles'], indent=4, sort_keys=True))

[
    "search-read",
    "entity-registry-admin",
    "data-store-write",
    "sys-admin-read",
    "prov-admin",
    "handle-write",
    "sys-admin-write",
    "handle-admin",
    "prov-read",
    "entity-registry-write",
    "default-roles-rrap",
    "prov-write",
    "entity-registry-read",
    "offline_access",
    "data-store-admin",
    "data-store-read",
    "handle-read",
    "uma_authorization",
    "sys-admin-admin"
]


### Test administration access

In [21]:
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-admin-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json()['roles'], indent=4, sort_keys=True))

Token validation failed due to error: Signature has expired.
Refreshing using refresh token

[
    "search-read",
    "entity-registry-admin",
    "data-store-write",
    "sys-admin-read",
    "prov-admin",
    "handle-write",
    "sys-admin-write",
    "handle-admin",
    "prov-read",
    "entity-registry-write",
    "default-roles-rrap",
    "prov-write",
    "entity-registry-read",
    "offline_access",
    "data-store-admin",
    "data-store-read",
    "handle-read",
    "uma_authorization",
    "sys-admin-admin"
]


[Return to Top](#toc)